In [ ]:
%matplotlib inline
from ROOT import TFile, TTree
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from root_numpy import root2array
from collections import OrderedDict

In [ ]:
filebase = '/Users/davidkaleko/larlite/UserDev/LowEnergyExcess/output/'
#filebase += '70KV/perfect_reco/nopi0alg/'
filebase += '70KV/perfect_reco/pi0alg_topoflist/'
#filebase += '70KV/perfect_reco/pi0alg_aftercosmics/'


filenames = OrderedDict([('nue','singleE_nue_selection_mc.root'),
                         ('numu','singleE_numu_selection_mc.root'),
                         ('nc','singleE_nc_selection_mc.root'),
                         ('cosmic','singleE_cosmic_selection_mc.root') ])#,
#                        ('lee','singleE_LEE_selection_mc.root')
#                        ])
treenames = { 'nue' : 'beamNuE',
             'cosmic' : 'cosmicShowers',
             'numu' : 'beamNuMu',
             'nc' : 'beamNC',
            'lee' : 'LEETree'}
labels = { 'nue' : 'Beam Intrinsic Nue',
         'cosmic' : 'CRY Cosmic, in-time',
         'numu' : 'Beam Intrinsic Numu',
         'nc' : 'Beam Intrinsic NC', 
         'lee' : 'Scaled Low Energy Excess'}
colors = { 'nue' : '#269729', #kGreen-2
         'numu' : '#4B4EAC', #kBlue-5
          'nc' : '#6B70F5', #kBlue-9
          'cosmic' : '#D12C2C', #kRed-3
          'lee' : '#E65C00' #orangish
          }
#binning = np.linspace(0.1,3,15)
binning = np.linspace(0.1,3,39)
#binning = np.linspace(0.1,3,15)
#binning = np.linspace(-1,1,39)
scaling_weights = { 'nue' : 6.6e20/(2.706e15*99600), #should be 99600, used 96000 for collab meeting
             'cosmic' : 2.52, #(211,000 ms total exposure)/(6.4ms * 13100 evts generated)
             'numu' : 6.6e20/(2.706e15*99600),
             'nc' : 6.6e20/(2.706e15*99600),
                  'lee' : 1}
#10cm from all sides
fidvolcut = '_x_vtx > 10 and _x_vtx < 246.35 and _y_vtx > -106.5 and _y_vtx < 106.5 and _z_vtx > 10 and _z_vtx < 1026.8'
defaultcut = '_longestTrackLen < 100.'
coscut = 'costheta > 0.5'

plot_variable = '_e_nuReco'#'ptoverp'

In [ ]:
dfs = OrderedDict()
for key, filename in filenames.iteritems():
    dfs.update( { key : pd.DataFrame( root2array( filebase + filename, treenames[key] ) ) } )
   

In [ ]:
for key in dfs.keys():
    #quick add column to df that is cosine of another column
    dfs[key]['ecostheta'] = np.cos(dfs[key]['_e_theta'])
    dfs[key]['nucostheta'] = np.cos(dfs[key]['_nu_theta'])
    dfs[key]['ptoverp'] = dfs[key]['_nu_pt']/dfs[key]['_nu_p']


In [ ]:
plt.figure(figsize=[10,6])
for key in dfs.keys():
    x = np.array(dfs[key]['ptoverp'])
    y = np.array(dfs[key]['_y_vtx'])
    plt.plot(x,y,'o',label=key)
plt.legend(loc=3)
plt.xlabel('Reconstructed Neutrino pt/p')
plt.ylabel('Reconstructed neutrino y-vertex')
plt.grid(True)

In [ ]:
#Let's do a logistic regression to classify if an event is nue or cosmic
#(todo: how is this different than a log likelihood?)
# y = 0 means cosmic, y = 1 means nue

#Build the feature matrix, X, from cosmic samples (with y = 0) and nue samples (with y = 1)
cosX = dfs['cosmic'][['ptoverp', '_y_vtx']].as_matrix()
cosX = np.insert(cosX,0,1,axis=1)
cosy = np.zeros((cosX.shape[0],1))
nueX = dfs['nue'][['ptoverp', '_y_vtx']].as_matrix()
nueX = np.insert(nueX,0,1,axis=1)
nuey = np.ones((nueX.shape[0],1))
X = np.vstack((cosX,nueX))
y = np.vstack((cosy,nuey))
m = y.size # number of training samples

In [ ]:
#Plot the data to make sure it looks good
def plotData():
    plt.figure(figsize=(10,6))
    plt.plot(cosX[:,1],cosX[:,2],'co',label='Cosmics')
    plt.plot(nueX[:,1],nueX[:,2],'ro',label='Nues')
    plt.xlabel('Reconstructed Neutrino pt/p')
    plt.ylabel('Reconstructed Neutrino y-vertex')
    plt.legend()
    plt.grid(True)
    
plotData()

In [ ]:
#Feature normalizing the columns (subtract mean, divide by standard deviation)
#Store the mean and std for later use
#Note don't modify the original X matrix, use a copy
stored_feature_means, stored_feature_stds = [], []
Xnorm = X.copy()
for icol in xrange(Xnorm.shape[1]):
    stored_feature_means.append(np.mean(Xnorm[:,icol]))
    stored_feature_stds.append(np.std(Xnorm[:,icol]))
    #Skip the first column
    if not icol: continue
    #Faster to not recompute the mean and std again, just used stored values
    Xnorm[:,icol] = (Xnorm[:,icol] - stored_feature_means[-1])/stored_feature_stds[-1]

In [ ]:
#Build the logistic regression:
from scipy.special import expit #Vectorized sigmoid function

#Hypothesis function and cost function for logistic regression
def h(mytheta,myX): #Logistic hypothesis function
    return expit(np.dot(myX,mytheta))

#Cost function, default lambda (regularization) 0
def computeCost(mytheta,myX,myy,mylambda = 0.): 
    """
    theta_start is an n- dimensional vector of initial theta guess
    X is matrix with n- columns and m- rows
    y is a matrix with m- rows and 1 column
    Note this includes regularization, if you set mylambda to nonzero
    For the first part of the homework, the default 0. is used for mylambda
    """
    #note to self: *.shape is (rows, columns)
    term1 = np.dot(-np.array(myy).T,np.log(h(mytheta,myX)))
    term2 = np.dot((1-np.array(y)).T,np.log(1-h(mytheta,myX)))
    regterm = (mylambda/2) * np.sum(np.dot(mytheta[1:].T,mytheta[1:])) #Skip theta0
    return float( (1./m) * ( np.sum(term1 - term2) + regterm ) )

In [ ]:
#Optimization functions:
#An alternative to OCTAVE's 'fminunc' we'll use some scipy.optimize function, "fmin"
#Note "fmin" does not need to be told explicitly the derivative terms
#It only needs the cost function, and it minimizes with the "downhill simplex algorithm."
#http://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.optimize.fmin.html
from scipy import optimize

def optimizeTheta(mytheta,myX,myy,mylambda=0.):
    result = optimize.fmin(computeCost, x0=mytheta, args=(myX, myy, mylambda), maxiter=400, full_output=True)
    return result[0], result[1]

In [ ]:
#Plug in a random initial theta and solve for the coefficients:
initial_theta = np.zeros((Xnorm.shape[1],1))
theta, mincost = optimizeTheta(initial_theta,Xnorm,y)
print theta

In [ ]:
#print h(np.array(theta),np.array([[1,0,-100]]))

In [ ]:
#Import necessary matplotlib tools for 3d plots
from mpl_toolkits.mplot3d import axes3d, Axes3D
from matplotlib import cm
import itertools

fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')

xvals = np.arange(-1,1,.05)
yvals = np.arange(-125,125,5)
myxs, myys, myzs = [], [], []
for david in xvals:
    for kaleko in yvals:
        myxs.append(david)
        myys.append(kaleko)
        testpoint = np.array([david,kaleko])
        #To "undo" feature normalization, we "undo" 1650 and 3, then plug it into our hypothesis
        testpointscaled = [(testpoint[x]-stored_feature_means[x+1])/stored_feature_stds[x+1] for x in xrange(len(testpoint))]
        testpointscaled.insert(0,1)
        myzs.append(h(np.array(theta),np.array([testpointscaled])))

scat = ax.scatter(myxs,myys,myzs,c=np.abs(myzs),cmap=plt.get_cmap('YlOrRd'))

plt.xlabel('Neutrino pt/p',fontsize=30)
plt.ylabel('Neutrino y-vertex',fontsize=30)
plt.title('Hypothesis (1 = nue, 0 = cosmic)',fontsize=30)
plt.show()

In [ ]:
#Hacky insert 0 in stored feature mean and 1 in stored feature stds
stored_feature_means[0]=0
stored_feature_stds[0]=1

In [ ]:
#Now that we have an optimized theta fit parameter vector, add hypothesis as a column to each df
for key in dfs.keys():
    testpoints = np.array([dfs[key]['ptoverp'],dfs[key]['_y_vtx']]).T
    testpoints = np.insert(testpoints,0,1,axis=1)
    testpointsscaled = (testpoints-stored_feature_means)/stored_feature_stds
    dfs[key]['hypothesis'] = h(np.array(theta),np.array([testpointsscaled])).T

In [ ]:
mybins = np.linspace(0,1,50)
plt.figure(figsize=(10,6))
plt.hist(dfs['cosmic']['hypothesis'],label='Cosmics',alpha=0.5,bins=mybins)
plt.hist(dfs['nue']['hypothesis'],label='Nue',alpha=0.5,bins=mybins)
plt.legend(loc=2)
plt.xlabel("Hypothesis Value (1 = Nue, 0 = Cosmic)")
plt.grid(True)

In [ ]:
def gen_histos(myquery=''):
    nphistos = OrderedDict()

    for key, df in dfs.iteritems():
        if key == 'lee': continue
        mydf = df.query(myquery) if myquery else df
        if key == 'cosmic':
            myweights = np.ones(mydf[plot_variable].shape[0])
        else:
            #NOTE: using ravel() seems to actually modify the dataframe itself
            #which causes weird behavior when this function is called repeatedly
            #so just cast the series as an array (as done below) instead.
            #I don't understand why ravel() modifies the DF.
            myweights = np.array(mydf['_weight'])#.ravel()
        myweights *= scaling_weights[key]
        nphistos.update( {key : np.histogram(mydf[plot_variable]/1000.,
                                     bins=binning,
                                     weights=myweights)} )
    return nphistos

In [ ]:
def plot_fullstack(myhistos):
    fig = plt.figure(figsize=(10,6))
    plt.grid(True)
    lasthist = 0
    for key, (hist, bins) in myhistos.iteritems():
        if key == 'lee': continue
        plt.bar(bins[:-1],hist,
                width=bins[1]-bins[0],
                color=colors[key],
                bottom = lasthist,
                edgecolor = 'k',
                label='%s: %d Events'%(labels[key],sum(hist)))
        lasthist += hist
    
    plt.xlim([binning[0],binning[-1]])
    #plt.ylim([0,400])
    plt.title('CCSingleE Stacked Backgrounds',fontsize=25)
    plt.ylabel('Events',fontsize=20)
    if plot_variable == '_e_nuReco':
        xstring = 'Reconstructed Neutrino Energy [GeV]' 
    else:
        xstring = 'pT/p Reconstructed Neutrino'
    plt.xlabel(xstring,fontsize=20)
    plt.legend()

In [ ]:
#nphistos = gen_histos('_is_fiducial==True and _longestTrackLen < 1')
plot_fullstack(gen_histos())
plot_fullstack(gen_histos('hypothesis > 0.5'))
#plot_fullstack(gen_histos(defaultcut))
#plot_fullstack(gen_histos(defaultcut + ' and ' + fidvolcut))
plot_fullstack(gen_histos(defaultcut + ' and ' + fidvolcut + ' and ' + 'hypothesis > 0.5'))
#plot_fullstack(gen_histos())
#plot_fullstack(gen_histos(fidvolcut))

In [ ]:
numu_MIDs = dfs['numu'].query(defaultcut + ' and ' + fidvolcut + ' and ' + 'hypothesis > 0.5')

numu_MIDs.hist('_parentPDG',bins=np.linspace(-100,200,300))
numu_MIDs['_is_fiducial']

In [ ]:
numu_MIDs.query('_parentPDG == 13 or _parentPDG == -13')

In [ ]:
\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

In [ ]:
\]